## 6. Deploy LVA & ML Solution to IoT Edge Device
Unless specified, all commands, cells below will be running on the development/deployment machine (not on the edge device)

### 6.1. Setting up IoT Edge deployment target

#### 6.1.1. Get global variables
We will read the previously stored variables

In [1]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

containerRegServiceName = get_key(envPath, "containerRegServiceName")
containerImageName = get_key(envPath, "containerImageName")
acrUserName = get_key(envPath, "acrUserName")
acrPassword = get_key(envPath, "acrPassword")
acrServer = get_key(envPath, "acrServer")
resourceGroupName = get_key(envPath, "resourceGroupName")
iotHubServiceName = get_key(envPath, "iotHubServiceName")
iotDeviceId = get_key(envPath, "iotDeviceId")
iotEdgeDeviceConnString = get_key(envPath, "iotEdgeDeviceConnString")
localContainerRegServiceName = get_key(envPath, "localContainerRegServiceName")

#### 6.1.2. Setup IoT Edge Device deployment variables

In [2]:
# # Set your RTSP Camera source details here.
# camAddress = "<rtsp://YOUR_IP_CAM_RTSP_ADDRESS>"
# camUsername = "<CAMERA_USERNAME>"
# camPassword = "<CAMERA_PASSWORD>"

# # Set to the scoring URI of the AI Extensibility module
# # DONT USE http://127.0.0.1 or http://localhost but use http://<MODULE_NAME>:<PORT_NUMBER>/<SCORING_PATH>
# scoringURI = "<SCORING_URI>"

# # Set the location of LVA module
# lvaModuleAddress = "<LVA_MODULE_ADDRESS>"

# # Set a local folder path that the debug output (analysed video frames, with overlayed analysis result, in jpeg file format) will be placed.
# hostDebugFolder = "<LOCAL_FOLDER_PATH_TO_DEBUG_OUTPUT>"

# # Set a value of how sensitive the motion should be. Pick one of three: low|medium|high
# motionSensitivity = "<MOTION_SENSITIVITY>"

# # Pick two frames per second from the camera stream and send only these two frames to the AI module... 
# # Set any number according to your Edge Device compute capability
# camFPSFilterRate = "<FPS_FILTER_RATE>"

# BELOW ARE SOME TEST VARIABLES WE USE. SET ABOVE VARIABLES WITH YOUR OWN VALUES AND DELETE ALL BELOW ASSIGNMENTS
camAddress = "rtsp://192.168.254.60/axis-media/media.amp"
# camAddress = "rtsp://92.168.254.60"
camUsername = "athena"
camPassword = "Python3rocks"
scoringURI = "http://" + containerImageName + ":5001/score"

lvaModuleAddress = "marketplace.azurecr.io/azure-media-services-preview/official/media_edge:1.3-linux-x64"

hostDebugFolder = "/tmp/aix_debug_cam"
motionSensitivity = "high"
camFPSFilterRate = "2"

imageTag = "v0.1.7"

#### 6.1.3. Create IoT Edge Device deployment configuration file from the template with our own variables
<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
> If you are in production, than uncomment the following line:  
>> *#deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', containerRegServiceName)*  
>> *#deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', containerRegServiceName+".azurecr.io")*  
>
> and comment the following line:  
>
>> *deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', localContainerRegServiceName)*  


In [5]:
# We have a deployment template file which needs to be updated with AI model's container registry address and credentials
file = open("../src/alpr/LVADeploymentTemplate.json")
deploymentTemplate = file.read()

# Update the variables
#deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', containerRegServiceName)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', localContainerRegServiceName)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', localContainerRegServiceName)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_USER_NAME', acrUserName)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_PASSWORD', acrPassword)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_MODULE_NAME', containerImageName)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_MODULE_TAG', imageTag)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_SCORING_URI', scoringURI)

deploymentTemplate = deploymentTemplate.replace('AIX_LVA_MODULE_ADDRESS', lvaModuleAddress)

deploymentTemplate = deploymentTemplate.replace('AIX_IP_CAM_ADDRESS', camAddress)
deploymentTemplate = deploymentTemplate.replace('AIX_IP_CAM_USER_NAME', camUsername)
deploymentTemplate = deploymentTemplate.replace('AIX_IP_CAM_PASSWORD', camPassword)
deploymentTemplate = deploymentTemplate.replace('AIX_IP_CAM_FPS_FILTER', camFPSFilterRate)

deploymentTemplate = deploymentTemplate.replace('AIX_HOST_DEBUG_FOLDER', hostDebugFolder)

deploymentTemplate = deploymentTemplate.replace('AIX_MOTION_SENSITIVITY', motionSensitivity)


# Save the resulting deployment file on disk, so we can pass it to deployment API (currently not supporting direct Json string input)
with open('../src/alpr/LVADeployment.json', 'wt', encoding='utf-8') as outputFile:
    outputFile.write(deploymentTemplate)
    
print(deploymentTemplate)

{
    "modulesContent": {
        "$edgeAgent": {
            "properties.desired": {
                "runtime": {
                    "settings": {
                        "minDockerVersion": "v1.25",
                        "registryCredentials": {
                            "mhregistry:55000": {
                                "username": "mhlva01contreg",
                                "password":"lPAd7o3eSDQs=QZbmX3ajI3v65dGlO/K",
                                "address":"mhregistry:55000"
                            }                            
                        }
                    },
                    "type": "docker"
                },
                "schemaVersion": "1.0",
                "systemModules": {
                    "edgeAgent": {
                        "settings": {
                            "image": "mcr.microsoft.com/azureiotedge-agent:1.0",
                            "createOptions": ""
                        },
                        "type"

### 6.2. Deploy to IoT Edge Target
In previous sections, we already set up our physical (or virtual on a VM) IoT Edge Device. Now we connect / match the physical device with the cloud based one on the IoTHub. Everything that we are pushing into IoTHub based edge device will will be cloned into the physical device (all IoT Edge modules, settings etc. defined in the manifest file)

In [130]:
# Install iot hub CLI extensions. To run below cells, we need iot edge extention
!az extension add --name azure-cli-iot-ext

Extension 'azure-cli-iot-ext' is already installed.


<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Warning] </span>  
Running below cell will reset and remove all previously deployed modules on the edge devices.

In [131]:
print(iotDeviceId)

mhlva01iotdevidlpr


In [4]:
# First Reset the edge device
!az iot edge set-modules --device-id $iotDeviceId --hub-name $iotHubServiceName --content "../src/alpr/LVAResetDeviceTemplate.json"

[
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "1lo/mEyA27JZ5ZAKJGH1m7k02M5LYjwkceX9DA3NLhk=",
        "secondaryKey": "cUjEcrniD3b6PhwQmy8yDfql6cJWOOKVmksv4LVGDpg="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null
      }
    },
    "cloudToDeviceMessageCount": 0,
    "connectionState": "Disconnected",
    "connectionStateUpdatedTime": "0001-01-01T00:00:00+00:00",
    "deviceId": "mhlva01iotdevidlpr",
    "etag": "ODg2MDIyNTU2",
    "generationId": "637231142544182398",
    "lastActivityTime": "0001-01-01T00:00:00+00:00",
    "managedBy": null,
    "moduleId": "$edgeAgent"
  },
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "Ucjz4ffJesBWQkiU0cRJr+j9gPpnpqn1DzWRZhFoBzY=",
        "secondaryKey": "AWRZCJTTbXNdKiynj3SoIcjxrrSArZHXfB+UcG7JC/A="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumb

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Warning] </span>  
Running below cell will deploy all modules (our AI extensibility module that we created, lva and all required ones) and settings to the edge devices.

In [6]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $iotDeviceId --hub-name $iotHubServiceName --content "../src/alpr/LVADeployment.json"

[
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "1lo/mEyA27JZ5ZAKJGH1m7k02M5LYjwkceX9DA3NLhk=",
        "secondaryKey": "cUjEcrniD3b6PhwQmy8yDfql6cJWOOKVmksv4LVGDpg="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null
      }
    },
    "cloudToDeviceMessageCount": 0,
    "connectionState": "Disconnected",
    "connectionStateUpdatedTime": "0001-01-01T00:00:00+00:00",
    "deviceId": "mhlva01iotdevidlpr",
    "etag": "ODg2MDIyNTU2",
    "generationId": "637231142544182398",
    "lastActivityTime": "0001-01-01T00:00:00+00:00",
    "managedBy": null,
    "moduleId": "$edgeAgent"
  },
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "Ucjz4ffJesBWQkiU0cRJr+j9gPpnpqn1DzWRZhFoBzY=",
        "secondaryKey": "AWRZCJTTbXNdKiynj3SoIcjxrrSArZHXfB+UcG7JC/A="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumb

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  

Once you run the above cell, or make new manifest deployment, run the following shell command on the **IoT Edge Device** to restart the IoT Edge Runtime
```bash
sudo systemctl restart iotedge
```

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
On the IoT Edge device, run the following shell command to see if all required modules pulled and running. Depending on your Internet connection speed, it may take some time to pull images into IoT Edge device.

```shell
sudo iotedge list
```

Running above command, you should see something similar to below 4 lines, telling that 4 IoTEdge modules (edgeAgent, edgeHub, mediaEdge and mkov01aimodule) pulled, up and running...

```
NAME             STATUS           DESCRIPTION      CONFIG
edgeAgent        running          Up 4 minutes     mcr.microsoft.com/azureiotedge-agent:1.0
mkov01aimodule   running          Up 4 minutes     mkregistry:55000/mkov01aimodule:latest
mediaEdge        running          Up 4 minutes     marketplace.azurecr.io/azure-media-services-preview/official/media_edge:1.2-linux-x64
edgeHub          running          Up 4 minutes     mcr.microsoft.com/azureiotedge-hub:1.0
```

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  

wait 1-2 minute to have IoT Edge Runtime up and running in stable mode. If things not working well, use the following shell command to restart it:

```shell
sudo iotedge restart mkov01aimodule
```

After all, you should see inference results going into logs and IoTHub which are explained in next section, section 9. If you dont observe any message flow or if you think that the pipeline is not working, you should try restarting just the LVA module:

```shell
sudo iotedge restart edgeMedia
```